In [7]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [26]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [17]:
!ls ../data

green_tripdata_2021-01.parquet  green_tripdata_2022-03.parquet
green_tripdata_2021-02.parquet  yellow_tripdata_2022-01.parquet
green_tripdata_2022-01.parquet  yellow_tripdata_2022-02.parquet
green_tripdata_2022-02.parquet


In [28]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet")

In [29]:
len(df_train), len(df_val)

(73908, 61921)

In [30]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [31]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [32]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [34]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
# Not working with the Lasso() model currently.
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

8.231436941224935

# This is the section on XGBoost.

In [35]:
import xgboost as xgb

In [36]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [37]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/the-greatest/Desktop/mlops-zoomcamp-private/notebooks/mlruns/2', creation_time=1685399287955, experiment_id='2', last_update_time=1685399287955, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [38]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [41]:
# This is a test!
with mlflow.start_run():
    mlflow.set_tag("developer", "Yingquan")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

In [39]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [42]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[19:00:46] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:12.71183                          
[1]	validation-rmse:9.04926                           
[2]	validation-rmse:7.63238                           
[3]	validation-rmse:7.11266                           
[4]	validation-rmse:6.90614                           
[5]	validation-rmse:6.81345                           
[6]	validation-rmse:6.76695                           
[7]	validation-rmse:6.74065                           
[8]	validation-rmse:6.72996                           
[9]	validation-rmse:6.72221                           
[10]	validation-rmse:6.71889                          
[11]	validation-rmse:6.71576                          
[12]	validation-rmse:6.71380                          
[13]	validation-rmse:6.71185                          
[14]	validation-

KeyboardInterrupt: 

In [43]:
params = {
    'learning_rate': 0.08853780665787961,
    'max_depth': 45,
    'min_child_weight': 1.9983648209360885,
    'objective': 'reg:linear',
    'reg_alpha': 0.009387754097696417,
    'reg_lambda': 0.18265859549779104,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)


2023/05/29 19:17:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd36eb2bb4e9c425285d4d0c95e73d21e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[19:17:16] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.60709
[1]	validation-rmse:18.17659
[2]	validation-rmse:16.88715
[3]	validation-rmse:15.72623
[4]	validation-rmse:14.68235
[5]	validation-rmse:13.74740
[6]	validation-rmse:12.90930
[7]	validation-rmse:12.16190
[8]	validation-rmse:11.49639
[9]	validation-rmse:10.90355
[10]	validation-rmse:10.37837
[11]	validation-rmse:9.91336
[12]	validation-rmse:9.50201
[13]	validation-rmse:9.13871
[14]	validation-rmse:8.81969
[15]	validation-rmse:8.53747
[16]	validation-rmse:8.29032
[17]	validation-rmse:8.07206
[18]	validation-rmse:7.88016
[19]	validation-rmse:7.71273
[20]	validation-rmse:7.56598
[21]	validation-rmse:7.43733
[22]	validation-rmse:7.32586
[23]	validation-rmse:7.22799
[24]	validation-rmse:7.14085
[25]	validation-rmse:7.06514
[26]	validation-rmse:6.99831
[2

2023/05/29 19:18:17 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/29 19:18:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/the-greatest/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
